In [10]:
import cirq, cirq_qubitization as cq
import cirq_qubitization
from typing import Sequence
import numpy as np
from collections import defaultdict

In [2]:
def get_hubbard_model_registers(x_dim: int, y_dim: int) -> cq.Registers:
    """1-D fermi hubbard model with N sites. Uses 2 * N spin orbitals.
    2 * i : Index of i'th spin orbital with spin-up.
    2 * i + 1 : Index of i'th spin orbital with spin-down.
    """
    N = x_dim * y_dim * 2
    return cq.Registers.build(
        control = 1,
        U = 1,
        V = 1,
        p_x = (x_dim - 1).bit_length(),
        p_y = (y_dim - 1).bit_length(),
        alpha = 1,
        q_x = (x_dim - 1).bit_length(),
        q_y = (y_dim - 1).bit_length(),
        beta = 1,
        target = N,
        ancilla = (x_dim - 1).bit_length() + (y_dim - 1).bit_length() + 2
    )

In [3]:
class SelectHubbard(cq.GateWithRegisters):
    def __init__(self, x_dim: int, y_dim: int): 
        # assert x_dim == y_dim
        self._registers = get_hubbard_model_registers(x_dim, y_dim)
    
    @property
    def registers(self) -> cq.Registers:
        return self._registers

    def decompose_from_registers(
        self, control, U, V, p_x, p_y, alpha, q_x, q_y, beta, target, ancilla
    ) -> cirq.OP_TREE:
        (control, U, V, alpha, beta) = control + U + V + alpha + beta
        yield cq.SelectedMajoranaFermionGate(len(p_x) + len(p_y) + 1, len(target), target_gate=cirq.Y).on_registers(
            control=control,
            selection=p_x + p_y + (alpha,),
            ancilla=ancilla[:-1],
            accumulator=ancilla[-1],
            target=target
        )
        yield cq.MultiTargetCSwap(len(p_x) + len(p_y) + 1).on_registers(
            control=V, 
            target_x=p_x + p_y + (alpha,), 
            target_y=q_x + q_y + (beta,)
        )
        yield cq.SelectedMajoranaFermionGate(len(q_x) + len(q_y) + 1, len(target), target_gate=cirq.X).on_registers(
            control=control,
            selection=q_x + q_y + (beta,),
            ancilla=ancilla[:-1],
            accumulator=ancilla[-1],
            target=target
        )
        yield cq.MultiTargetCSwap(len(p_x) + len(p_y) + 1).on_registers(
            control=V, 
            target_x=p_x + p_y + (alpha,), 
            target_y=q_x + q_y + (beta,)
        )
        yield cirq.S(control) ** -1
        yield cirq.CZ(control, U)
        yield cq.ApplyGateToLthQubit(len(q_x) + len(q_y) + 1, len(target), 
            lambda n:cirq.Z if n&1 else cirq.I, control_bitsize=2).on_registers(
            control=[control, V],
            selection=q_x + q_y + (beta,),
            ancilla=ancilla[:-1],
            target=target
        )
        

In [4]:
select_gate = SelectHubbard(1, 4)
registers = select_gate.registers
qubit_map = registers.get_named_qubits()
qubits = list(q for v in registers.get_named_qubits().values() for q in v)
op = select_gate.on_registers(**qubit_map)
print(cirq.Circuit(cirq.decompose_once(op)).to_text_diagram(qubit_order=qubits))

control: ────@────────────@─────S^-1───@───@─────
             │            │            │   │
U: ──────────┼────────────┼────────────@───┼─────
             │            │                │
V: ──────────┼─────@──────┼─────@──────────@─────
             │     │      │     │          │
p_y0: ───────In────×(x)───┼─────×(x)───────┼─────
             │     │      │     │          │
p_y1: ───────In────×(x)───┼─────×(x)───────┼─────
             │     │      │     │          │
alpha: ──────In────×(x)───┼─────×(x)───────┼─────
             │     │      │     │          │
q_y0: ───────┼─────×(y)───In────×(y)───────In────
             │     │      │     │          │
q_y1: ───────┼─────×(y)───In────×(y)───────In────
             │     │      │     │          │
beta: ───────┼─────×(y)───In────×(y)───────In────
             │            │                │
target0: ────ZY───────────ZX───────────────I─────
             │            │                │
target1: ────ZY───────────ZX───────────────Z─────


In [15]:
hubbard_hamiltonian = cq.hubbard_qubitization.qubit_hamiltonian_fermi_hubbard(
        x_dimension=4,
        y_dimension=1,
        tunneling=1,
        coulomb=4,
        periodic=True,
)
d = defaultdict(lambda: [])
for ps in hubbard_hamiltonian:
    d[ps.coefficient].append(ps.copy(coefficient=1))
for coefficient, terms in d.items():
    print(coefficient)
    print(*sorted(terms, key=lambda t: str(t)), sep="\n")

(-0.5+0j)
+IIIIIXZX
+IIIIIYZY
+IIIIXZXI
+IIIIYZYI
+IIIXZXII
+IIIYZYII
+IIXZXIII
+IIYZYIII
+IXZXIIII
+IXZZZZZX
+IYZYIIII
+IYZZZZZY
+XZXIIIII
+XZZZZZXI
+YZYIIIII
+YZZZZZYI
(4+0j)
+IIIIIIII
(-1+0j)
+IIIIIIIZ
+IIIIIIZI
+IIIIIZII
+IIIIZIII
+IIIZIIII
+IIZIIIII
+IZIIIIII
+ZIIIIIII
(1+0j)
+IIIIIIZZ
+IIIIZZII
+IIZZIIII
+ZZIIIIII
